# PoliGPT - Conversação RAG

In [12]:
import os

os.environ["AZURESEARCH_FIELDS_ID"] = "chunk_id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "chunk"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "vector"

In [13]:
from langchain_core.utils import get_from_env
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_community.vectorstores.azuresearch import AzureSearch, AzureSearchVectorStoreRetriever

In [14]:
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

## Parameters

In [16]:
# Conectar ao Azure Key Vault
kv_uri = "https://kv-poligpt-dev-eastus2.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

In [17]:
OPENAI_API_KEY = client.get_secret("openai-api-key").value
COMPLETION_MODEL = "gpt-4o-2024-08-06"
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_DIMENSIONS = 3072

AZURE_SEARCH_ENDPOINT = client.get_secret("azure-search-endpoint").value
AZURE_SEARCH_ADMIN_KEY = client.get_secret("azure-search-admin-key").value
SEARCH_INDEX_NAME = "poligpt-index"

## Setup

### LLM

In [18]:
llm = ChatOpenAI(
    model=COMPLETION_MODEL,
    api_key=OPENAI_API_KEY,
    streaming=True,
    temperature=0.3
)

### Embeddings

In [19]:
embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY, 
    dimensions=EMBEDDING_DIMENSIONS
)

### Vector Store

In [20]:
vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name=SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
    search_type="hybrid"
)

In [ ]:
# # Teste do vector store
# docs = vector_store.hybrid_search(
#     query="Como me inscrevo numa materia?",
#     k=3
# )

# docs

[Document(metadata={'chunk_id': '52fc867f789a_7e37d055112566ce114ae3d34b4ec12f_pages_0', 'title': 'Microsoft Word - PoliMoodle-Inscricao-em-disciplina', 'parent_id': '7e37d055112566ce114ae3d34b4ec12f', 'last_modified': '09/14/2021 00:00:00 +00:00', 'keywords': 'tutoriais', '@search.score': 0.027756938710808754, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, page_content='STI Setor de Tecnologia da Informação \nEscola Politécnica \nUFRJ\n\nPoliMoodle – Inscrição em Disciplina\n\nSTI Poli UFRJ – http://www.sti.poli.ufrj.br – sti@poli.uftj.br V2020-08-19\n\nPara se inscrever como aluno em uma disciplina no PoliMoodle, você deve seguir os passos abaixo:\n\n1. Acesse o ambiente PoliMoodle (http://www.moodle.poli.ufrj.br), informando a sua conta e a sua senha.\n\n2. A área “Meus cursos” mostra as disciplinas que você está inscrito. Clique em "Todos os cursos" para se \ninscrever em outras disciplinas.\n\n3. Na nova tela, você poderá navegar pelos Depar

In [ ]:
# retriever = AzureSearchVectorStoreRetriever(
#     name="AzureSearchRetriever",
#     vectorstore=vector_store,
#     search_type="hybrid",
#     k=3
# )

## RAG Agent

In [58]:
# Incorporar persistência em memória
memory = MemorySaver()

In [ ]:
@tool(response_format="content_and_artifact")
def retriever_tool(query: str):
    """Busca e retorna informações relacionadas a questões acadêmicas ou institucionais sobre a Escola Politécnica da UFRJ."""
    retrieved_docs = vector_store.hybrid_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
tools = [retriever_tool]

In [ ]:
agent_executor = create_react_agent(
    model=llm, 
    tools=tools, 
    checkpointer=memory
)

In [61]:
config = {"configurable": {"thread_id": "def234"}}

input_message = (
    "A politecnica ja recebeu algum prêmio?"
)

for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

A politecnica ja recebeu algum prêmio?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_3HgXJcKHuWDTMSqILiN2PAdQ)
 Call ID: call_3HgXJcKHuWDTMSqILiN2PAdQ
  Args:
    query: prêmios recebidos pela Escola Politécnica
================================= Tool Message =================================
Name: retrieve

Source: {'chunk_id': 'e1ff2efac568_f602c08267838493473dce29662b965c_pages_0', 'parent_id': 'f602c08267838493473dce29662b965c', 'last_modified': '11/25/2022 00:00:00 +00:00', 'keywords': 'comunicacao,informativos', 'title': None, '@search.score': 0.021271489560604095, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
Content: Estudantes da Escola Politécnica recebem premiação do 
Crea-RJ
LEIA
Governador do Rio sanciona projeto de lei de alunos da 
Politécnica-UFRJ 
LEIA
Professor da Engenharia Naval receb

In [56]:
memory.get(config=config)

---

## Testes com Agente 

In [21]:
retriever = AzureSearchVectorStoreRetriever(
    name="AzureSearchRetriever",
    vectorstore=vector_store,
    search_type="hybrid",
    k=3
)

In [83]:
memory = MemorySaver()

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever = retriever,
    name = "buscar_poli_info",
    description = "Busca e retorna informações relacionadas a questões acadêmicas ou institucionais sobre a Escola Politécnica da UFRJ.",
)

tools = [retriever_tool]

In [84]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = SystemMessage(
    content="""
    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados 
    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo
    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes 
    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.
    """
)

agent_executor = create_react_agent(
    model=llm, 
    tools=tools, 
    state_modifier=system_message,
    checkpointer=memory
)
agent_executor.step_timeout = 10

In [ ]:
query = "Meu nome é Barbara."

config = {"configurable": {"thread_id": "test-thread"}}
messages = agent_executor.invoke(input={"messages": [("user", query)]}, config=config)

In [45]:
print(messages["messages"][-1].content)

Prazer em conhecê-la, Barbara! Como posso ajudar você hoje?


---

In [85]:
from langchain_core.messages import trim_messages
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

chat_history = [
    system_message,
    HumanMessage(content="Oi! Meu nome é Julia."),
    AIMessage(content="Ola!"),
    HumanMessage(content="Como voce esta?"),
    AIMessage(content="Bem, obrigado!"),
]

trimmed_history = trim_messages(
    chat_history,
    strategy="last",
    token_counter=llm,
    max_tokens=180,
    start_on="human",
    end_on="ai",
    include_system=True,
    allow_partial=False
)

In [90]:
query = "Tem noticias relevantes recentes sobre a escola politecnica?"

chat_messages = trimmed_history + [HumanMessage(content=query)]

config = {"configurable": {"thread_id": "test-thread-2"}}

messages = agent_executor.invoke(
    input={"messages": chat_messages}, 
    config=config
)

print(messages["messages"][-1].content)

Aqui estão algumas notícias recentes sobre a Escola Politécnica da UFRJ:

1. **PetroTeam da Politécnica-UFRJ é bicampeão do Petrobowl Championship**: Estudantes do curso de Engenharia de Petróleo conquistaram o 1º lugar no Petrobowl Championship.

2. **Nova diretoria do Clube de Engenharia para triênio 2024/2027 toma posse**: A nova diretoria eleita para o triênio 2024-2027 do Clube de Engenharia tomou posse recentemente.

3. **Professor da Politécnica participa de evento global sobre proteção contra raios**: O professor Antonio Carlos Siqueira de Lima do Departamento de Engenharia Elétrica participou de um importante evento global dedicado à ciência e tecnologia de proteção contra raios.

4. **Medalha de ouro nos Jogos Universitários Brasileiros**: O aluno Lucas Elbert do curso de Engenharia Eletrônica e Computação conquistou medalha de ouro durante a edição praia dos Jogos Universitários Brasileiros.

5. **Minerva Náutica fica entre as cinco melhores equipes em competição nacional**:

In [91]:
messages

{'messages': [SystemMessage(content='\n    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados \n    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo\n    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes \n    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.\n    ', additional_kwargs={}, response_metadata={}, id='fe17a7db-b9b0-4b03-8430-8b3dcac01dfe'),
  HumanMessage(content='Oi! Meu nome é Julia.', additional_kwargs={}, response_metadata={}, id='04862b72-363e-416d-a7b4-66df9576862a'),
  AIMessage(content='Ola!', addit

In [92]:
agent_executor.get_state(config=config)

StateSnapshot(values={'messages': [SystemMessage(content='\n    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados \n    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo\n    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes \n    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.\n    ', additional_kwargs={}, response_metadata={}, id='fe17a7db-b9b0-4b03-8430-8b3dcac01dfe'), HumanMessage(content='Oi! Meu nome é Julia.', additional_kwargs={}, response_metadata={}, id='04862b72-363e-416d-a7b4-66df9576862a'), AIMessage(cont